In [7]:
import numpy as np
import json 
import pandas as pd 
from scipy.special import gamma, kv
from typing import List, Dict, Any
from scipy.optimize import minimize
import torch

# 1. Read Data, Convert to DataFrame

In [8]:
def convert_json_to_df(json_file_path:str):
    with open(json_file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    in_sample_transactions = data["transactions"]["in_sample_transactions"]
    out_sample_transactions = data["transactions"]["out_of_sample_transactions"]
    product_labels = data['product_labels']

    in_sample_transactions = pd.DataFrame(in_sample_transactions)
    out_sample_transactions = pd.DataFrame(out_sample_transactions)
    
    # rename 'prodcut' to 'choice' 
    in_sample_transactions.rename(columns={'product':'choice'}, inplace=True)
    out_sample_transactions.rename(columns={'product':'choice'}, inplace=True)
    product_labels = pd.DataFrame(
        list(product_labels.items()), columns=["product_id", "product_name"]
    )
    return in_sample_transactions, out_sample_transactions,product_labels

In [9]:
def convert_list_to_one_hot(transaction:list,d):
    one_hot = np.zeros(d)
    for item in transaction:
        one_hot[item-1] = 1
    return one_hot

def convert_to_one_hot(transactions:pd.DataFrame,d):
    transactions["offered_product_one_hot"] = transactions['offered_products'].apply(lambda x : convert_list_to_one_hot(x,d))
    transactions['choice_one_hot'] = transactions['choice'].apply(lambda x: np.zeros(d) if x == 0 else  convert_list_to_one_hot([x],d))
    return transactions



In [10]:
instance_id = 5
in_sample_transactions, out_sample_transactions ,items= convert_json_to_df(f"hotel_json/instance_{instance_id}.json")
d = len(items)
datasize = len(in_sample_transactions)
in_sample_transactions = convert_to_one_hot(in_sample_transactions,d)
out_sample_transactions = convert_to_one_hot(out_sample_transactions,d)

in_sample_transactions

,offered_products,choice,offered_product_one_hot,choice_one_hot
0,"[0, 1, 2, 3, 4, 5, 6]",5,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
1,"[0, 1, 2, 3, 4, 5, 6]",0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,"[0, 1, 2, 3, 4, 5, 6]",0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,"[0, 1, 2, 3, 4, 5, 6]",0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,"[0, 1, 2, 3, 4, 5, 6]",0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...,...,...
995,"[0, 2, 4, 5]",2,"[0.0, 1.0, 0.0, 1.0, 1.0, 1.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
996,"[0, 2, 4, 5]",0,"[0.0, 1.0, 0.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
997,"[0, 2, 4, 5]",0,"[0.0, 1.0, 0.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
998,"[0, 2, 4, 5]",0,"[0.0, 1.0, 0.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


# 2. Kernel Implementation





Matrix-valued **Matern kernel** $\tilde{\boldsymbol{\mathsf{k}}}$ can be 

$$
\boldsymbol{\mathsf{k}}(\boldsymbol{e}_{S},\boldsymbol{e}_{S'}) = \boldsymbol{K} \otimes \mathsf{k}_{m}(\boldsymbol{e}_{S}, \boldsymbol{e}_{S'})
$$
where $\boldsymbol{K}$ is a positive semi-definite matrix. Then 

$$
\tilde{\mathsf{k}}^{ij}(\boldsymbol{e}_{S},\boldsymbol{e}_{S'}) = K_{ij} \times\sigma^{2} \frac{2^{1-\nu}}{\Gamma(\nu)} \left( \sqrt{ 2\nu }  \frac{\left\| \boldsymbol{e}_{S} - \boldsymbol{e}_{S'} \right\|_{2}  }{\ell} \right) K_{\nu} \left( \sqrt{ 2\nu } \frac{\left\| \boldsymbol{\boldsymbol{e}_{S}-\boldsymbol{e}_{S'}} \right\|_{2}  }{\ell} \right)
$$
Add constraint, 

$$
\mathsf{k}^{ij}(S, S') = \mathbb{1}(i \in \boldsymbol{e}_{S})\cdot \mathbb{1}(j \in \boldsymbol{e}_{S'}) \cdot   \tilde{\mathsf{k}}^{ij}(\boldsymbol{e}_{S}, \boldsymbol{e}_{S'})
$$


Same with Gaussian kernel.



In [11]:

K = np.ones((d,d))

def generate_scalar_matern_kernel(length_scale:float, nu:float, sigma:float):
    """
    Input: kernel parameters and index (i,j)
    generate base scalar-valued Matern kernel at (i,j) 
    return {k_m}_ij
    """
    def kernel(x1:np.ndarray, x2:np.ndarray):

        dist = np.linalg.norm(x1 - x2)
        
        if dist == 0:
            return sigma**2

        # calculate the factor
        factor = (2 ** (1 - nu)) / gamma(nu)
        scaled_dist  = np.sqrt(2 * nu) * dist / length_scale
        result = sigma**2 * factor * (scaled_dist**nu) * kv(nu, scaled_dist)
        return result
    return kernel

def generate_scalar_gaussian_kernel(length_scale:float, sigma:float):
    """
    Input: kernel parameters and index (i,j)
    generate base scalar-valued Gaussian kernel at (i,j)
    return {k_g}_ij
    """
    def kernel(x1:np.ndarray, x2:np.ndarray):
        
        dist = np.linalg.norm(x1 - x2)
        return sigma**2 * np.exp(-dist**2 / (2 * length_scale**2))
    return kernel

def generate_matrix_matern_kernel(length_scale, nu, sigma):
    """
    generate a matrix-valued Matern kernel 
    """
    scalar_matern_kernel = generate_scalar_matern_kernel(length_scale, nu, sigma)
    def kernel(x1:np.ndarray,x2:np.ndarray):
        dim = x1.shape[0]
        result = np.zeros((dim,dim))
        for i in range(dim):
            for j in range(dim):
                if x1[i] == 0 or x2[j] == 0:
                    result[i,j] = 0
                else:
                    result[i,j] = scalar_matern_kernel(x1,x2) * K[i,j]
        return result
    return kernel


def generate_matrix_gaussian_kernel(length_scale, sigma):
    """
    generate a matrix-valued Gaussian kernel 
    """
    scalar_gaussian_kernel = generate_scalar_gaussian_kernel(length_scale, sigma)
    def kernel(x1:np.ndarray,x2:np.ndarray):
        dim = x1.shape[0]
        result = np.zeros((dim,dim))
        for i in range(dim):
            for j in range(dim):
                if x1[i] == 0 or x2[j] == 0:
                    result[i,j] = 0
                else:
                    result[i,j] = scalar_gaussian_kernel(x1, x2) * K[i,j]
        return result
    return kernel


scalar_matern_kernel = generate_scalar_matern_kernel(1, 1, 1)
scalar_gaussian_kernel = generate_scalar_gaussian_kernel(1, 1)
matrix_matern_kernel = generate_matrix_matern_kernel(1, 1, 1)
matrix_gaussian_kernel = generate_matrix_gaussian_kernel(1, 1)




    
    


In [12]:
from tqdm import tqdm


total_iterations = datasize * datasize
with tqdm(total=total_iterations, desc="Overall Progress", unit="iteration") as pbar:
    K_kernel = np.zeros((datasize, datasize, d,d))  

    for i in range(datasize):
        for j in range(datasize):

            K_kernel[i, j] = matrix_matern_kernel(
                in_sample_transactions["offered_product_one_hot"][i],
                in_sample_transactions["offered_product_one_hot"][j],
            )

            pbar.update(1)  

Overall Progress: 100%|██████████| 1000000/1000000 [02:31<00:00, 6593.25iteration/s]


In [13]:
K_kernel = torch.tensor(K_kernel,dtype=torch.float32)
K_kernel.shape

torch.Size([1000, 1000, 6, 6])

# 3. Solve

In [21]:
alphaset = torch.randn((datasize, d), dtype=torch.float32, requires_grad=True)
lambda_ = 0.001

def objective(alphaset: torch.Tensor):
    U = torch.zeros((datasize, d), dtype=torch.float32)
    U = torch.einsum("ijkl, jk -> il", K_kernel, alphaset)

    l = loss(U)
    r = reg(alphaset)

    return l + lambda_ * r



def loss(U: torch.Tensor):

    loss_value = 0.0
    for i in range(datasize):

        p_vec = torch.zeros((d, 1), dtype=torch.float32)

        hS_i = torch.tensor(
            in_sample_transactions.iloc[i]["offered_product_one_hot"],
            dtype=torch.float32,
        ).view(-1, 1)

        y_i = torch.tensor(
            in_sample_transactions.iloc[i]["choice_one_hot"], dtype=torch.float32
        ).view(-1, 1)

        utility_hSi = U[i].view(-1, 1)
        exp_utility = torch.exp(utility_hSi)
        sum_exp_utility = torch.sum(exp_utility) + 1

        for j in range(d):

            if hS_i[j] == 1:

                p_vec[j] = torch.exp(utility_hSi[j]) / sum_exp_utility
            else:

                p_vec[j] = 0

        loss_value += cross_entropy_loss(p_vec, y_i)
    return loss_value / datasize


def cross_entropy_loss(p_vec: torch.Tensor, y_vec: torch.Tensor):
    # print('y_vec',y_vec)
    for i in range(d):
        if y_vec[i] == 1:
            return -torch.log(p_vec[i])
    return 0


def squared_loss(p_vec: torch.Tensor, y_vec: torch.Tensor):
    return torch.sum((p_vec - y_vec) ** 2)


def reg(alphaset: torch.Tensor):

    alphaset = alphaset.unsqueeze(2)  # 增加一个维度以便执行einsum

    # 正则化项的计算
    result = torch.einsum("ikd,ijkl,jle->", alphaset, K_kernel, alphaset)

    return result

def compute_gradient():
    objective_value = objective(alphaset)
    objective_value.backward()  # 计算梯度
    return alphaset.grad

optimizer = torch.optim.Adam([alphaset], lr=0.01)
for epoch in range(1000):
    optimizer.zero_grad()  
    loss_value = objective(alphaset)
    loss_value.backward()  
    optimizer.step()  

    print(f"Epoch {epoch}, Loss: {loss_value.item()}")


Epoch 0, Loss: 3.8568854331970215
Epoch 1, Loss: 3.4104156494140625
Epoch 2, Loss: 2.3018760681152344
Epoch 3, Loss: 1.984501600265503
Epoch 4, Loss: 2.287419319152832
Epoch 5, Loss: 1.674678921699524
Epoch 6, Loss: 1.88460111618042
Epoch 7, Loss: 2.1403815746307373
Epoch 8, Loss: 2.08719539642334
Epoch 9, Loss: 1.771384358406067
Epoch 10, Loss: 1.4352787733078003
Epoch 11, Loss: 1.3189496994018555
Epoch 12, Loss: 1.7327287197113037
Epoch 13, Loss: 1.3278980255126953
Epoch 14, Loss: 1.219597578048706
Epoch 15, Loss: 1.2232204675674438
Epoch 16, Loss: 1.272365927696228
Epoch 17, Loss: 1.2892640829086304
Epoch 18, Loss: 1.2386466264724731
Epoch 19, Loss: 1.1419211626052856
Epoch 20, Loss: 1.046735167503357
Epoch 21, Loss: 0.9826368689537048
Epoch 22, Loss: 0.9481101036071777
Epoch 23, Loss: 0.9336551427841187
Epoch 24, Loss: 0.9191620349884033
Epoch 25, Loss: 0.8764467239379883
Epoch 26, Loss: 0.8262518644332886
Epoch 27, Loss: 0.7780554890632629
Epoch 28, Loss: 0.7367410659790039
Epoch 